In [1]:
import os
import numpy as np

import data_preprocessing
import neural_metrics
import utils

In [7]:
data_str = 'imagenet'
binary_num_epochs = 5
main_model_name = 'dinov2_vits14'

preprocessor = data_preprocessing.DataPreprocessor(data_str=data_str)
test_fine_true = preprocessor.test_true_fine_data
test_coarse_true = preprocessor.test_true_coarse_data
train_fine_true = preprocessor.train_true_fine_data
train_coarse_true = preprocessor.train_true_coarse_data

binary_results_path = 'binary_results'
binary_l_strs = list({f for f in os.listdir('binary_results') if f.startswith(f'{data_str}_{main_model_name}')})
len(binary_l_strs)

In [17]:
for name in binary_l_strs:
    curr_path = fr'{binary_results_path}/{name}'
    curr_data = np.load(curr_path)
    binary_num_epochs = 5
    l_str = name.split(f'e{binary_num_epochs - 1}_')[-1].replace('.npy', '')
    is_fine = l_str in preprocessor.fine_grain_classes_str
    l_str_index = getattr(preprocessor, 
                          'fine_grain_classes_str' if is_fine else 'coarse_grain_classes_str').index(l_str) 
    test = len(curr_data) == len(test_coarse_true)
    true_values = (test_fine_true if is_fine else test_coarse_true) if test else (train_fine_true if is_fine else train_coarse_true)
    binary_true = np.where(true_values == l_str_index, 1, 0)
    accuracy, f1, precision, recall = neural_metrics.get_individual_metrics(pred_data=curr_data,
                                                                            true_data=binary_true)
    print(f'class: {l_str}; '
          f'accuracy: {accuracy:.2f}, '
          f'f1: {f1:.2f}, '
          f'precision: {precision:.2f}, '
          f'recall: {recall:.2f}')
    
    assert f1 > 0.7